In [23]:
import numpy as np
import os
import cv2
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import pairwise_distances

# Load images and labels from directories
data_dir = "../dataset/fer2013plus/train"
emotions = ['anger', 'contempt', 'disgust', 'fear', 'happiness', 'neutral', 'sadness', 'surprise']
X = []
y = []

# Iterate through directories for each emotion
for i, emotion in enumerate(emotions):
    emotion_dir = os.path.join(data_dir, emotion)
    for img_name in os.listdir(emotion_dir):
        img_path = os.path.join(emotion_dir, img_name)
        # Read image and convert to grayscale
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        # Resize image to a fixed size if necessary
        # img = cv2.resize(img, (width, height))
        # Flatten image pixel values and append to X
        X.append(img.flatten())
        # Append emotion label to y
        y.append(i)  # Using the index as label

# Convert lists to NumPy arrays
X = np.array(X)
y = np.array(y)

# Initialize Nearest Neighbors model
nn_model = NearestNeighbors(n_neighbors=2)  # Considering only the nearest neighbor

# Fit the model
nn_model.fit(X)

# Compute pairwise distances between samples
distances = pairwise_distances(X)

# Initialize lists to store intra-class and extra-class distances
intra_class_distances = []
extra_class_distances = []

# Iterate through each sample in the dataset
for i in range(len(X)):
    # Get label of current sample
    label = y[i]
    
    # Find the indices of samples with the same label
    same_class_indices = np.where(y == label)[0]
    
    # Find the indices of samples with different labels
    different_class_indices = np.where(y != label)[0]
    
    # Compute distances to nearest neighbors from the same class
    same_class_distances = distances[i, same_class_indices]
    # Exclude the distance to itself (always 0)
    same_class_distances = same_class_distances[same_class_distances != 0]
    # Append the minimum distance to intra_class_distances
    intra_class_distances.append(np.min(same_class_distances))
    
    # Compute distances to nearest neighbors from a different class
    different_class_distances = distances[i, different_class_indices]
    # Append the minimum distance to extra_class_distances
    extra_class_distances.append(np.min(different_class_distances))

# Convert lists to NumPy arrays
intra_class_distances = np.array(intra_class_distances)
extra_class_distances = np.array(extra_class_distances)

# Calculate N1 metric
intra_extra= np.sum(intra_class_distances) / np.sum(extra_class_distances)
N1 = intra_extra / (1 + intra_extra)

# # Calculate N2 metric
# N2 = np.sum(y != y[nn_model.kneighbors(X, 2, return_distance=False)[:, 1]]) / len(y)

# print("N1 metric:", N1)
# print("N2 metric:", N2)

# # Calculate F1 metric
# max_fdr = np.max(intra_class_distances / extra_class_distances)
# F1 = 1 / (1 + max_fdr)
# print("F1 metric:", F1)

# # Calculate F2 metric
# max_fdr = np.max(intra_class_distances / extra_class_distances)
# F2 = 1 / (1 + max_fdr**2)
# print("F2 metric:", F2)


N1 metric: 0.4948506934952714
N2 metric: 0.5155710561544423
F1 metric: 0.008226889679952219
F2 metric: 6.880449415981515e-05


In [24]:
#calculate how many image in contempt folder
contempt_dir = os.path.join(data_dir, 'contempt')
contempt_count = len(os.listdir(contempt_dir))
print("Number of images in contempt folder:", contempt_count)

neutral_dir = os.path.join(data_dir, 'neutral')
neutral_count = len(os.listdir(neutral_dir))
print("Number of images in neutral folder:", neutral_count)

Number of images in contempt folder: 165
Number of images in neutral folder: 10308


In [27]:
cn = neutral_count
cp = contempt_count

ir = cn/cp

Ob = cp * (ir - 1)

print (str(ir))
print (str(Ob))

Op = Ob * N1

print (str(Op))

62.472727272727276
10143.0
5019.270584122538


In [1]:
a = 10308 - 7528
b = a * 0.49485

print(b)

1375.683
